In [74]:
import pandas as pd

# Анализ данных об аренде велосипедов в Чикаго

In [10]:
bikes_q1 = pd.read_csv(bikes_q1, parse_dates=['start_time','end_time'])

Для начала, возьмем данные только за Q1 <br>


### Структура данных
`trip_id` — id поездки <br>
`start_time` — Дата и время начала поездки<br>
`end_time` — Дата и время конца поездки<br>
`bikeid` — id велосипеда<br>
`tripduration` — продолжительность поездки в минутах<br>
`from_station_id` — id станции начала поездки<br>
`from_station_name` — название пункта отправления<br>
`to_station_id` — id пункта прибытия<br>
`to_station_name` — название пункта прибытия<br>
`usertype` — тип пользователя<br>
`gender` — пол (если subscriber)<br>
`birthyear` — год рождения (если subscriber)<br>

In [3]:
bikes_q1.head()

,trip_id,start_time,end_time,bikeid,tripduration,from_station_id,from_station_name,to_station_id,to_station_name,usertype,gender,birthyear
0,17617135,2018-01-22 20:04:31,2018-01-22 20:11:53,1131,442.0,471,Francisco Ave & Foster Ave,468,Budlong Woods Library,Subscriber,Female,1949.0
1,17897619,2018-03-16 19:47:59,2018-03-16 20:04:00,6146,961.0,296,Broadway & Belmont Ave,253,Winthrop Ave & Lawrence Ave,Subscriber,Male,1988.0
2,17881307,2018-03-14 18:49:20,2018-03-14 18:54:38,3847,318.0,260,Kedzie Ave & Milwaukee Ave,503,Drake Ave & Fullerton Ave,Subscriber,Male,1987.0
3,17881130,2018-03-14 18:33:48,2018-03-14 19:07:40,1483,"2,032.0",199,Wabash Ave & Grand Ave,199,Wabash Ave & Grand Ave,Subscriber,Male,1990.0
4,17686289,2018-02-05 17:39:14,2018-02-05 17:46:13,6391,419.0,596,Benson Ave & Church St,605,University Library (NU),Subscriber,Male,1992.0


In [4]:
bikes_q1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174215 entries, 0 to 174214
Data columns (total 12 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   trip_id            174215 non-null  int64  
 1   start_time         174215 non-null  object 
 2   end_time           174215 non-null  object 
 3   bikeid             174215 non-null  int64  
 4   tripduration       174215 non-null  object 
 5   from_station_id    174215 non-null  int64  
 6   from_station_name  174215 non-null  object 
 7   to_station_id      174215 non-null  int64  
 8   to_station_name    174215 non-null  object 
 9   usertype           174215 non-null  object 
 10  gender             164161 non-null  object 
 11  birthyear          164236 non-null  float64
dtypes: float64(1), int64(4), object(7)
memory usage: 15.9+ MB


In [6]:
bikes_q1.duplicated().sum()

0

In [8]:
bikes_q1.dtypes
# так как у столбцов с датой стоит тип object, приведем к формату DateTime при чтении csv файла

trip_id                int64
start_time            object
end_time              object
bikeid                 int64
tripduration          object
from_station_id        int64
from_station_name     object
to_station_id          int64
to_station_name       object
usertype              object
gender                object
birthyear            float64
dtype: object

#### Перед тем как сделать .resample(), немного подготовим данные. Поместим колонку start_time в качестве индексов и сохраните изменения в исходный датасет.

In [12]:
bikes_q1 = bikes_q1.set_index('start_time')

In [14]:
bikes_q1.head()

,trip_id,end_time,bikeid,tripduration,from_station_id,from_station_name,to_station_id,to_station_name,usertype,gender,birthyear
start_time,,,,,,,,,,,
2018-01-22 20:04:31,17617135,2018-01-22 20:11:53,1131,442.0,471,Francisco Ave & Foster Ave,468,Budlong Woods Library,Subscriber,Female,1949.0
2018-03-16 19:47:59,17897619,2018-03-16 20:04:00,6146,961.0,296,Broadway & Belmont Ave,253,Winthrop Ave & Lawrence Ave,Subscriber,Male,1988.0
2018-03-14 18:49:20,17881307,2018-03-14 18:54:38,3847,318.0,260,Kedzie Ave & Milwaukee Ave,503,Drake Ave & Fullerton Ave,Subscriber,Male,1987.0
2018-03-14 18:33:48,17881130,2018-03-14 19:07:40,1483,"2,032.0",199,Wabash Ave & Grand Ave,199,Wabash Ave & Grand Ave,Subscriber,Male,1990.0
2018-02-05 17:39:14,17686289,2018-02-05 17:46:13,6391,419.0,596,Benson Ave & Church St,605,University Library (NU),Subscriber,Male,1992.0


#### 1. В данных имеется как дата аренды, так и её точное время начала и окончания с точностью до секунд. Применим метод pd.resample() и агрегируем данные по дням. Посмотрим максимальное число аренд за день

In [23]:
bikes_q1.resample('1d').trip_id.count().sort_values(ascending=False)

start_time
2018-02-27    4196
2018-03-28    3640
2018-02-26    3422
2018-02-28    3412
2018-03-15    3245
              ... 
2018-02-10     364
2018-02-11     331
2018-02-09     300
2018-02-04     279
2018-01-01     191
Name: trip_id, Length: 90, dtype: int64

**Вывод:** максимально число аренд за день  было 27 февраля и составляло 4196

#### 2. Посмотрим на распределение количества аренд для разных групп пользователей (usertype) — customers и subscribers в данных за апрель

Сделаем ресемпл по дням для каждой группы и *в качестве ответа укажем число аренд за 18 апреля, сделанных пользователями типа Subscribe*

In [27]:
bikes_apr = pd.read_csv(bikes_apr, parse_dates=['start_time','end_time'])

In [32]:
bikes_apr.set_index('start_time', inplace=True)

In [33]:
bikes_apr.head()

,trip_id,end_time,bikeid,tripduration,from_station_id,from_station_name,to_station_id,to_station_name,usertype,gender,birthyear
start_time,,,,,,,,,,,
2018-04-01 00:17:00,18000534,2018-04-01 00:22:53,1323,353.0,130,Damen Ave & Division St,69,Damen Ave & Pierce Ave,Subscriber,Male,1991.0
2018-04-01 00:20:00,18000536,2018-04-01 00:26:22,2602,382.0,121,Blackstone Ave & Hyde Park Blvd,351,Cottage Grove Ave & 51st St,Subscriber,Female,1992.0
2018-04-01 00:23:19,18000538,2018-04-01 00:35:01,4213,702.0,31,Franklin St & Chicago Ave,180,Ritchie Ct & Banks St,Subscriber,Male,1985.0
2018-04-01 00:24:46,18000540,2018-04-01 00:44:23,6401,"1,177.0",596,Benson Ave & Church St,517,Clark St & Jarvis Ave,Subscriber,Male,1974.0
2018-04-01 00:26:04,18000541,2018-04-01 00:31:05,6333,301.0,145,Mies van der Rohe Way & Chestnut St,24,Fairbanks Ct & Grand Ave,Subscriber,Male,1984.0


In [44]:
bikes_april = bikes_apr.query("usertype == 'Subscriber'")\
                        .trip_id\
                        .resample('1d')\
                        .count()\
                        .to_frame()\
                        .reset_index()
bikes_april.head()

,start_time,trip_id
0,2018-04-01,549
1,2018-04-02,1865
2,2018-04-03,1276
3,2018-04-04,1501
4,2018-04-05,1637


In [47]:
bikes_april.query("start_time == '2018-04-18'")

,start_time,trip_id
17,2018-04-18,1453


**Вывод:** 18 апреля было 1453 аренды

#### 3. Посмотрим на данные за период с апреля по декабрь.

Объединим сэмплы данных за нужные месяцы в один общий датасет bikes. Сделаем преобразование по дням для каждой группы пользователей (usertype), затем выберем **дни, в которые число аренд, сделанных customers, было больше, чем у subscribers**

In [88]:
#данные за каждый месяц лежат в отдельной таблице, необходимо их объединить
bikes = []

In [89]:
links = [
    "bikes_q1_sample.csv",
    "bikes_q2_sample_apr.csv",
    "bikes_q2_sample_may.csv",
    "bikes_q2_sample_jun.csv",
    "bikes_q3_sample_july.csv",
    "bikes_q3_sample_aug.csv",
    "bikes_q3_sample_sep.csv",
    "bikes_q4_sample_oct.csv",
    "bikes_q4_sample_nov.csv",
    "bikes_q4_sample_dec.csv",
]

In [90]:
for link in links:
    df = pd.read_csv(link, parse_dates=['start_time','end_time'])
    bikes.append(df)

In [91]:
bikes = pd.concat(bikes)

In [94]:
bikes.shape

(1138996, 12)

In [97]:
bikes.head()

,trip_id,start_time,end_time,bikeid,tripduration,from_station_id,from_station_name,to_station_id,to_station_name,usertype,gender,birthyear
0,17617135,2018-01-22 20:04:31,2018-01-22 20:11:53,1131,442.0,471,Francisco Ave & Foster Ave,468,Budlong Woods Library,Subscriber,Female,1949.0
1,17897619,2018-03-16 19:47:59,2018-03-16 20:04:00,6146,961.0,296,Broadway & Belmont Ave,253,Winthrop Ave & Lawrence Ave,Subscriber,Male,1988.0
2,17881307,2018-03-14 18:49:20,2018-03-14 18:54:38,3847,318.0,260,Kedzie Ave & Milwaukee Ave,503,Drake Ave & Fullerton Ave,Subscriber,Male,1987.0
3,17881130,2018-03-14 18:33:48,2018-03-14 19:07:40,1483,"2,032.0",199,Wabash Ave & Grand Ave,199,Wabash Ave & Grand Ave,Subscriber,Male,1990.0
4,17686289,2018-02-05 17:39:14,2018-02-05 17:46:13,6391,419.0,596,Benson Ave & Church St,605,University Library (NU),Subscriber,Male,1992.0


In [98]:
bikes.set_index('start_time', inplace=True)

In [102]:
bikes_day = bikes.groupby('usertype') \
                    .trip_id \
                    .resample('1d') \
                    .count()
bikes_day

usertype    start_time
Customer    2018-01-01      7
            2018-01-02      1
            2018-01-03     13
            2018-01-04     13
            2018-01-05     10
                         ... 
Subscriber  2018-12-27    684
            2018-12-28    872
            2018-12-29    350
            2018-12-30    385
            2018-12-31    352
Name: trip_id, Length: 730, dtype: int64

In [107]:
bikes_day_usertype = bikes_day.unstack(level = 'usertype')
bikes_day_usertype

usertype,Customer,Subscriber
start_time,,
2018-01-01,7,184
2018-01-02,1,758
2018-01-03,13,1084
2018-01-04,13,1099
2018-01-05,10,994
...,...,...
2018-12-27,28,684
2018-12-28,65,872
2018-12-29,23,350


In [112]:
(bikes_day_usertype.Customer > bikes_day_usertype.Subscriber)\
        .to_frame(name='Customer_more_than_Subscriber')\
        .query("Customer_more_than_Subscriber == True")

,Customer_more_than_Subscriber
start_time,
2018-05-27,True
2018-09-02,True


**Вывод:** число аренд, сделанных customers, было больше, чем у subscribers 27 мая и 2 сентября, все остальные дни аренд было больше у subscribers

#### 4. Более подробно посмотрим на летний период. Какой наиболее популярный пункт назначения?
В переменную bikes_summer сохраним наблюдения с 1 июня по 31 августа. Затем запишем в top_destination наиболее популярный пункт назначения. Сагрегируем данные по дням и определим, **в какой день в полученный пункт (top_destination) было совершено меньше всего поездок**

In [116]:
bikes.reset_index(inplace=True)
bikes_summer = bikes.query("start_time >= '2018-06-01' & start_time <= '2018-08-31'")
top_destination  = bikes_summer.to_station_name.value_counts().idxmax()
top_destination_bikes = bikes_summer.query("to_station_name == @top_destination")
top_destination_bikes = top_destination_bikes.set_index('start_time')
bad_day = top_destination_bikes.resample(rule = 'D').to_station_name.count().idxmin()
bad_day = bad_day.strftime('%Y-%m-%d')

In [123]:
print('Наиболее популярный пункт назначения -', top_destination, ',', 
      bad_day, 'было совершено меньше всего поездок в данный пункт')

Наиболее популярный пункт назначения - Streeter Dr & Grand Ave , 2018-06-21 было совершено меньше всего поездок в данный пункт


#### 5. Куда больше всего ездят на выходных?

In [124]:
bikes_summer['day_of_the_week'] = bikes_summer.start_time.dt.day_name()

C:\Users\mi\AppData\Local\Temp/ipykernel_4132/415584076.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bikes_summer['day_of_the_week'] = bikes_summer.start_time.dt.day_name()


In [126]:
bikes_summer.head()

,start_time,trip_id,end_time,bikeid,tripduration,from_station_id,from_station_name,to_station_id,to_station_name,usertype,gender,birthyear,day_of_the_week
354916,2018-06-01 00:04:40,18709077,2018-06-01 00:06:47,3155,127.0,128,Damen Ave & Chicago Ave,214,Damen Ave & Grand Ave,Subscriber,Female,1978.0,Friday
354917,2018-06-01 00:06:08,18709080,2018-06-01 00:24:18,2807,"1,090.0",258,Logan Blvd & Elston Ave,69,Damen Ave & Pierce Ave,Customer,NaN,NaN,Friday
354918,2018-06-01 00:08:01,18709086,2018-06-01 00:32:55,2737,"1,494.0",337,Clark St & Chicago Ave,225,Halsted St & Dickens Ave,Customer,Male,1988.0,Friday
354919,2018-06-01 00:09:02,18709091,2018-06-01 00:19:21,6089,619.0,210,Ashland Ave & Division St,56,Desplaines St & Kinzie St,Subscriber,Male,1987.0,Friday
354920,2018-06-01 00:09:28,18709092,2018-06-01 00:14:44,2352,316.0,240,Sheridan Rd & Irving Park Rd,303,Broadway & Cornelia Ave,Subscriber,Male,1997.0,Friday


In [127]:
bikes_summer.groupby(['day_of_the_week', 'from_station_name']).size().nlargest(10).to_frame()

,,0
day_of_the_week,from_station_name,
Saturday,Streeter Dr & Grand Ave,2814
Sunday,Streeter Dr & Grand Ave,2119
Saturday,Lake Shore Dr & Monroe St,1536
Tuesday,Canal St & Adams St,1535
Wednesday,Canal St & Adams St,1493
Saturday,Lake Shore Dr & North Blvd,1475
Wednesday,Streeter Dr & Grand Ave,1428
Monday,Canal St & Adams St,1424
Thursday,Canal St & Adams St,1412


**Выводы:** <br>
    - Наиболее популярный пункт назначения в субботу – Streeter Dr & Grand Ave<br>
    - Второй по числу поездок в субботу – Lake Shore Dr & North Blvd<br>
    - В будние дни больше всего поездок совершается до Streeter Dr & Grand Ave